# Parte 4 - Sequence embeddings

In [ ]:
from sentence_transformers import SentenceTransformer

# model = SentenceTransformer("distiluse-base-multilingual-cased-v2")
# model = SentenceTransformer("neuralmind/bert-base-portuguese-cased")
model = SentenceTransformer("rufimelo/Legal-BERTimbau-sts-base")  # português brasileiro
# model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")  # mais leve e eficiente

In [ ]:
import pandas as pd

hinos_analise: pd.DataFrame = pd.read_pickle("..\\assets\\hinos_analise_tokens.pkl")

In [ ]:
import numpy as np

# cria embeddings diretamente para cada hino (texto inteiro)
embeddings = model.encode(hinos_analise["texto_limpo"].tolist(), show_progress_bar=True)
X_sent = np.array(embeddings)

print(X_sent.shape)  # (n_hinos, 512)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(X_sent)

# hinos mais semelhantes ao hino 443
similarities = list(enumerate(similarity_matrix[443]))
similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

print("Mais parecidos com o hino 443:")
for idx, score in similarities[1:6]:
    print(f"Hino {idx}: {hinos_analise['nome'].iloc[idx]} → similaridade {score:.3f}")

In [ ]:
import umap
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
X_umap = umap_model.fit_transform(X_sent)

# número de clusters (experimente, ex.: 4 ou 6)
kmeans = KMeans(n_clusters=6, random_state=42, n_init=10)
hinos_analise["cluster"] = kmeans.fit_predict(X_sent)

hinos_analise["umap1"] = X_umap[:, 0]
hinos_analise["umap2"] = X_umap[:, 1]

plt.figure(figsize=(10, 8))
sns.scatterplot(
    data=hinos_analise, x="umap1", y="umap2", hue="cluster", palette="tab10", s=80
)
plt.title("Mapa dos hinos com Sentence Embeddings (UMAP)")
plt.show()

In [ ]:
query = "a palavra é alimento para a alma"
query_vec = model.encode([query])

scores = cosine_similarity(query_vec, X_sent)[0]
top_idx = np.argsort(scores)[::-1][:10]

print("Top hinos para a busca:")
for i in top_idx:
    print(f"Hino {i}: {hinos_analise['nome'].iloc[i]} → score {scores[i]:.3f}")

# Tópicos

In [ ]:
from bertopic import BERTopic

# Criar o modelo BERTopic
topic_model = BERTopic(embedding_model=model)

# Treinar modelo
topics, probs = topic_model.fit_transform(hinos_analise["texto_limpo"])

# Associar tópicos ao DataFrame
hinos_analise["topic"] = topics

print("\nDistribuição de tópicos por hino:")
print(hinos_analise[["nome", "topic"]])

# Mostrar os tópicos descobertos
print("\nTópicos extraídos:")
for topic_num in set(topics):
    if (
        topic_num != -1
    ):  # -1 significa "outlier" (documento não encaixou em nenhum cluster)
        palavras = topic_model.get_topic(topic_num)
        print(f"Tópico {topic_num}: {palavras}")

In [ ]:
topic_model.visualize_topics()
topic_model.visualize_documents(hinos_analise["texto_limpo"])